<img src="https://farm66.staticflickr.com/65535/49055715328_092031af74_o.png"  width="150" />

### CBPF - Centro Brasileiro de Pesquisas Físicas

# Projeto final do curso de Análise de big data e Astroinformática

### João Paulo Correia de França
### contato: joao.contato505@gmail.com


### Professor: Clécio R. de Bom

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *
import os, fnmatch
from PIL import Image

%matplotlib inline

In [ ]:
# deep learning results
deep_results = pd.read_csv('./results_catalog.txt')
deep_results.head()

In [ ]:
# cutting data
deep_results = deep_results[deep_results["Einstein_Radius_actual"]<2.25]

In [ ]:
# get metrics, such as mean bias, median bias and sigma68
def get_metrics(true,predicted):
    """
    def function which give metrics on data
    true: list/array
        true values
    predicted: list/array
        predicted values
    """
    # delta einstein angle
    deltatheta=true-predicted
    # bias over mean
    mean_bias=np.mean(deltatheta)
    # sigma 16 and 84 percentiles
    p16=np.percentile(deltatheta,15.85)
    p84=np.percentile(deltatheta,84.05)
    # sigma 68 
    sigma68=0.5*(p84-p16)
    # bias over median
    median_bias=np.percentile(deltatheta,50)
    return mean_bias,median_bias,sigma68

In [ ]:
# results of deep learning analysis predicted x true analysis
types = ['Inception', 'Sequential', 'ResNet', 'ResNetXt']

plt.rcParams.update({"font.size": 13})

fig, ax = plt.subplots(1, 4, figsize=(16, 4), sharex=True, sharey=True)
    
for i in range(0, len(types)):
    ax[i].scatter(deep_results['Einstein_Radius_actual'], deep_results['Einstein_Radius_'+types[i]], c=np.abs(deep_results['Einstein_Radius_'+types[i]]-deep_results['Einstein_Radius_actual'])/deep_results['Einstein_Radius_actual'], ec='k')
    ax[i].plot(np.linspace(min(deep_results['Einstein_Radius_actual']), max(deep_results['Einstein_Radius_actual'])), np.linspace(min(deep_results['Einstein_Radius_actual']), max(deep_results['Einstein_Radius_actual'])), 'r', lw=2.5)
    ax[i].set_ylim(min(deep_results['Einstein_Radius_actual']), 2.25);
    ax[i].set_xlabel('true $θ_E$')

    if i == 0:
        ax[i].set_ylabel('predicted $θ_E$')

In [ ]:
# metrics output
for i in range(0, len(types)):
    mean_bias, median_bias, sigma68=get_metrics(deep_results['Einstein_Radius_actual'], deep_results['Einstein_Radius_'+types[i]])
    print('====================='+types[i]+'=====================')
    print('σ68: '+str(round(sigma68,3)), ', Mean Bias: '+str(round(mean_bias,3)), ', Median Bias: '+str(round(median_bias,3)))

## Outlier detection on bayesian analysis

In [ ]:
def find(pattern, path):
    """
    def function to get specific files on a folder

    pattern: str
        file extension
    path: str
        path of the file
    """
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [ ]:
# duplicated output path, picking-up only pair
fit_images = find('subplot_fit_imaging.png', './output/')
del fit_images[1::2]

In [ ]:
print('Total inverse modeling files on path:', len(fit_images))

In [ ]:
# visual way to detect outliers, option list for widgets
optionslist = []
for i in range(0,len(fit_images)):
    optionslist.append(i)

In [ ]:
fit_images[0][9:-75]

In [ ]:
fit_images[0]

In [ ]:
def update_graph(i):
    """
    def function to update our visual graph
    i: int
        option, system
    """
    print(fit_images[i][9:-75])
    fig, ax = plt.subplots(1, 1, figsize=(16, 16), sharex=True, sharey=True)
    img=np.array(Image.open(fit_images[i]))
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    
i = widgets.Dropdown(options=optionslist, value=15, description='Component:', disabled=False)
widgets.interactive(update_graph,i = i)

In [ ]:
# outliers detected
outliers = [343, 330, 860, 474, 276, 87, 769, 805, 1215, 121, 871, 1095, 955, 1082, 1337]
print('Number of detected outliers: ', len(outliers))

In [ ]:
# Catalog simulation parameters (true values)
lenses_DESc = pd.read_csv('./lenses_DESc.txt',
                          sep=' ',
                          comment='#',
                          names=['id', 'zl', 'zs', 'b', 'sig_v', 'ql', 'rl', 'lens_g', 'lens_r', 'lens_i', 'xs', 'ys', 'qs', 'ps', 'rs', 'source_g', 'source_r', 'source_i', 'mu_s', 'g_band_coadd_seeing', 'g_band_coadd_signal_to_noise', 'r_band_coadd_seeing', 'r_band_coadd_signal_to_noise', 'i_band_coadd_seeing'])
lenses_DESc.head()

In [ ]:
# picking up results obtained after an outlier detection, here we compare with nominal values
theta_einstein = []
theta_einstein_chains = []
index = []
for i in range(0, len(fit_images)):
    if int(fit_images[i][9:-75]) not in outliers:
        # getting the system index
        idx=int(fit_images[i][9:-75])
        index.append(idx)
        # getting model results, here we have used pickle to recover all the model
        model_results_path=find('samples.pickle', './output/'+str(idx))[0]
        result = pd.read_pickle(model_results_path)
        # MCMC chain
        chain = pd.DataFrame(np.array(result.results["samples"]), columns=result.names, dtype=float)
        burnin = int(len(chain)*0.7) # burnin of 10%
        # getting the percentiles of 1 sigma
        theta_value_inf, theta_value, theta_value_sup = np.percentile(chain['galaxies_lens_mass_einstein_radius'][burnin:], [15.85, 50., 84.05])
        # passing through a list from dictionary
        theta_einstein_chains.append(np.array(chain['galaxies_lens_mass_einstein_radius']))
        theta_einstein.append({'inf':theta_value_inf, 'mean':theta_value, 'sup':theta_value_sup})

In [ ]:
# variation over the real value

In [ ]:
disc = []
for i in range(0, len(theta_einstein)):
    disc.append(theta_einstein[i]['mean']- float(lenses_DESc[lenses_DESc['id']==index[i]]['b']))

In [ ]:
plt.rcParams.update({"font.size": 13})
plt.hist(disc, range=(-0.3, 0.4), ec='k', bins=10, density=True)
plt.xlabel('$θ_{E,m}$-$θ_{E,t}$');

In [ ]:
# graph to compare nominal values to predicted ones in 1 sigma
plt.rcParams.update({"font.size": 10})

# rows and columns
rows = 3
columns = 5

theta_einstein_reshaped = np.array(theta_einstein).reshape(rows, columns)
index_reshaped = np.array(index).reshape(rows, columns)

fig, ax = plt.subplots(rows, columns, figsize=(16, 8), sharey=True)
fig.tight_layout() 
for i in range(0, rows):
    for j in range(0, columns):
        ax[i][j].errorbar(theta_einstein_reshaped[i][j]['mean'], 0, xerr=[[theta_einstein_reshaped[i][j]['mean']-theta_einstein_reshaped[i][j]['inf']], [theta_einstein_reshaped[i][j]['sup']-theta_einstein_reshaped[i][j]['mean']]], fmt="o", c="b")
        #print(float(lenses_DESc[lenses_DESc['id']==index[i]]['b']))
        ax[i][j].plot(float(lenses_DESc[lenses_DESc['id']==index_reshaped[i][j]]['b']), 1, 'ko')
        ax[i][j].set_yticks([])
        ax[i][j].set_title(str(index_reshaped[i][j]))

In [ ]:
# widget image options
image_model_optionlist = []
for i in range(0, len(index)):
        image_model_optionlist.append(i)

In [ ]:
# only the good ones
fit_images_ok = []
for i in range(0, len(fit_images)):
    if int(fit_images[i][9:-75]) in index:
        fit_images_ok.append(fit_images[i])

In [ ]:
def update_graph(i):
    """
    def function to update our visual graph
    i: int
        option, system
    """
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    
    fig.suptitle('$θ_{E, t}$:  '+str(round(float(lenses_DESc[lenses_DESc['id']==index[i]]['b']), 2))+'   $θ_{E, m}$: '+str(round(theta_einstein[i]['mean'], 2)) + ' $\pm$ (' +str(round(theta_einstein[i]['mean']-theta_einstein[i]['inf'], 2))+ ', '+str(round(theta_einstein[i]['sup']-theta_einstein[i]['mean'], 2))+')', fontsize=20)
    
    img=np.array(Image.open(fit_images_ok[i]))
    
    ax[0].imshow(img)
    ax[0].set_title('Original Image')
    ax[0].set_xlim(160, 440)
    ax[0].set_ylim(390, 120)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].spines['left'].set_visible(False)
    
    ax[1].imshow(img)
    ax[1].set_title('Model Image')
    ax[1].set_xlim(870, 1150)
    ax[1].set_ylim(390, 120)
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['bottom'].set_visible(False)
    ax[1].spines['left'].set_visible(False)

    
i = widgets.Dropdown(options=image_model_optionlist, value=0, description='Component:', disabled=False)
widgets.interactive(update_graph,i = i)